In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns   
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
ny20160702=pd.read_csv('072.csv')

C:\Users\lenovo\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (43,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
ny_zip = ny20160702[['price','host_response_rate','host_has_profile_pic',
                   'host_identity_verified','longitude','latitude',
                   'zipcode','property_type','room_type','accommodates',
                   'bathrooms', 'bedrooms', 'beds', 'bed_type',
                   'extra_people','minimum_nights','review_scores_rating',
                   'cancellation_policy']]

In [4]:
ny_zip = ny_zip.dropna()

In [5]:
dummies1 = pd.get_dummies(ny_zip.host_has_profile_pic)
dummies2 = pd.get_dummies(ny_zip.host_identity_verified)
dummies3 = pd.get_dummies(ny_zip.property_type)
dummies4 = pd.get_dummies(ny_zip.room_type)
dummies5 = pd.get_dummies(ny_zip.bed_type)
dummies6 = pd.get_dummies(ny_zip.cancellation_policy)

In [6]:
ny_zip = pd.concat([ny_zip, dummies1, dummies2, dummies3,  dummies4,  dummies5,  dummies6 ], axis=1)
ny_zip.head()

,price,host_response_rate,host_has_profile_pic,host_identity_verified,longitude,latitude,zipcode,property_type,room_type,accommodates,...,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,no_refunds,strict,super_strict_30,super_strict_60
0,65.0,1.0,t,t,-73.78,40.84,10464.00,House,Private room,2.0,...,0,0,0,1,0,1,0,0,0,0
1,125.0,1.0,t,t,-73.79,40.85,10464.00,Apartment,Entire home/apt,3.0,...,0,0,0,1,1,0,0,0,0,0
2,79.0,1.0,t,t,-73.78,40.85,10464.00,House,Entire home/apt,4.0,...,0,0,0,1,1,0,0,0,0,0
4,59.0,1.0,t,t,-73.79,40.85,10464.00,House,Private room,4.0,...,0,0,0,1,0,1,0,0,0,0
5,99.0,1.0,t,f,-73.79,40.85,10464.00,Apartment,Entire home/apt,4.0,...,0,0,0,1,0,0,0,1,0,0


In [7]:
ny_zip.drop(ny_zip.columns[[ 2, 3, 4, 5, 7, 8]], axis=1, inplace=True)
ny_zip.head()

,price,host_response_rate,zipcode,accommodates,bathrooms,bedrooms,beds,bed_type,extra_people,minimum_nights,...,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,no_refunds,strict,super_strict_30,super_strict_60
0,65.0,1.0,10464.00,2.0,1.0,1.0,1.0,Real Bed,10.0,1.0,...,0,0,0,1,0,1,0,0,0,0
1,125.0,1.0,10464.00,3.0,1.0,1.0,1.0,Real Bed,0.0,3.0,...,0,0,0,1,1,0,0,0,0,0
2,79.0,1.0,10464.00,4.0,1.0,1.0,2.0,Real Bed,10.0,1.0,...,0,0,0,1,1,0,0,0,0,0
4,59.0,1.0,10464.00,4.0,1.0,1.0,1.0,Real Bed,10.0,1.0,...,0,0,0,1,0,1,0,0,0,0
5,99.0,1.0,10464.00,4.0,1.0,1.0,2.0,Real Bed,10.0,3.0,...,0,0,0,1,0,0,0,1,0,0


In [8]:
ny_zip.drop('bed_type', axis=1, inplace=True)

In [9]:
ny_zip.drop('cancellation_policy', axis=1, inplace=True)


In [10]:
ny_zip.zipcode = pd.to_numeric(ny_zip.zipcode,errors = 'coerce')
ny_zip = ny_zip.dropna()
ny_zip.zipcode.apply(lambda x: int(x))

0        10464
1        10464
2        10464
4        10464
5        10464
6        10464
7        10464
8        10464
9        10467
10       10469
11       10469
12       10469
13       10469
14       10469
15       10469
16       10467
19       10469
20       10467
21       10469
22       10469
23       10469
24       10467
25       10469
26       10467
27       10467
28       10469
30       11105
32       11105
33       11105
35       11105
         ...  
38755    11211
38756    11211
38758    11222
38760    11211
38762    11211
38763    11206
38764    11211
38765    11211
38766    11211
38768    11211
38770    11211
38771    11206
38772    11249
38773    11249
38774    11211
38775    11249
38776    11222
38777    11206
38778    11211
38780    11211
38782    11249
38783    11211
38785    11249
38788    10705
38789    10065
38791    11581
38799    10304
38801    10304
38802    11694
38809    10467
Name: zipcode, dtype: int64

In [11]:
zipcode = pd.read_excel('zipcode.xlsx')

In [12]:
zipcode = zipcode.set_index(zipcode['zipcode'], drop = True)

In [13]:
zipcode['boro_number'] =0

In [14]:
def findboro(zip_code):
    b = zipcode.ix[zip_code, 'boro']
    return b

In [15]:
boro = pd.Series.tolist(findboro(ny_zip.zipcode))

In [16]:
ny_zip['boro'] = boro

In [17]:
ny_zip.head()

,price,host_response_rate,zipcode,accommodates,bathrooms,bedrooms,beds,extra_people,minimum_nights,review_scores_rating,...,Futon,Pull-out Sofa,Real Bed,flexible,moderate,no_refunds,strict,super_strict_30,super_strict_60,boro
0,65.0,1.0,10464.0,2.0,1.0,1.0,1.0,10.0,1.0,92.0,...,0,0,1,0,1,0,0,0,0,Bronx
1,125.0,1.0,10464.0,3.0,1.0,1.0,1.0,0.0,3.0,92.0,...,0,0,1,1,0,0,0,0,0,Bronx
2,79.0,1.0,10464.0,4.0,1.0,1.0,2.0,10.0,1.0,98.0,...,0,0,1,1,0,0,0,0,0,Bronx
4,59.0,1.0,10464.0,4.0,1.0,1.0,1.0,10.0,1.0,97.0,...,0,0,1,0,1,0,0,0,0,Bronx
5,99.0,1.0,10464.0,4.0,1.0,1.0,2.0,10.0,3.0,97.0,...,0,0,1,0,0,0,1,0,0,Bronx


In [18]:
ny_zip = ny_zip.reset_index()

In [19]:
ny_zip['boro_number'] = 0 

In [20]:
for i in range(len(ny_zip)):
    b = ny_zip.ix[i,'boro']
    if b == 'Manhattan':
        b_n=0
    elif b =='Brooklyn':
        b_n = 1
    elif b =='Bronx':
        b_n = 2
    elif b =='Queens':
        b_n = 3
    elif b == 'Staten':
        b_n =4
    else:
        b_n = NaN
    ny_zip.boro_number[i] = b_n

In [21]:
ny_zip.head()


,index,price,host_response_rate,zipcode,accommodates,bathrooms,bedrooms,beds,extra_people,minimum_nights,...,Pull-out Sofa,Real Bed,flexible,moderate,no_refunds,strict,super_strict_30,super_strict_60,boro,boro_number
0,0,65.0,1.0,10464.0,2.0,1.0,1.0,1.0,10.0,1.0,...,0,1,0,1,0,0,0,0,Bronx,0
1,1,125.0,1.0,10464.0,3.0,1.0,1.0,1.0,0.0,3.0,...,0,1,1,0,0,0,0,0,Bronx,0
2,2,79.0,1.0,10464.0,4.0,1.0,1.0,2.0,10.0,1.0,...,0,1,1,0,0,0,0,0,Bronx,0
3,4,59.0,1.0,10464.0,4.0,1.0,1.0,1.0,10.0,1.0,...,0,1,0,1,0,0,0,0,Bronx,0
4,5,99.0,1.0,10464.0,4.0,1.0,1.0,2.0,10.0,3.0,...,0,1,0,0,0,1,0,0,Bronx,0


In [25]:
del ny_zip['boro']

In [26]:
del ny_zip['index']

In [30]:
del ny_zip['zipcode']

In [31]:
ny_zip.head()

,price,host_response_rate,accommodates,bathrooms,bedrooms,beds,extra_people,minimum_nights,review_scores_rating,f,...,Futon,Pull-out Sofa,Real Bed,flexible,moderate,no_refunds,strict,super_strict_30,super_strict_60,boro_number
0,65.0,1.0,2.0,1.0,1.0,1.0,10.0,1.0,92.0,0,...,0,0,1,0,1,0,0,0,0,0
1,125.0,1.0,3.0,1.0,1.0,1.0,0.0,3.0,92.0,0,...,0,0,1,1,0,0,0,0,0,0
2,79.0,1.0,4.0,1.0,1.0,2.0,10.0,1.0,98.0,0,...,0,0,1,1,0,0,0,0,0,0
3,59.0,1.0,4.0,1.0,1.0,1.0,10.0,1.0,97.0,0,...,0,0,1,0,1,0,0,0,0,0
4,99.0,1.0,4.0,1.0,1.0,2.0,10.0,3.0,97.0,0,...,0,0,1,0,0,0,1,0,0,0


In [32]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
Y = ny_zip.price

X = ny_zip.iloc[:,1:]

In [33]:
import statsmodels.api as sm
model = sm.OLS(Y.astype(float),X.astype(float))
results = model.fit()
results.summary()


C:\Users\lenovo\Anaconda2\lib\site-packages\statsmodels\base\model.py:978: RuntimeWarning: invalid value encountered in divide
  return self.params / self.bse
C:\Users\lenovo\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in greater
  cond1 = (scale > 0) & (x > self.a) & (x < self.b)
C:\Users\lenovo\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1748: RuntimeWarning: invalid value encountered in less
  cond1 = (scale > 0) & (x > self.a) & (x < self.b)
C:\Users\lenovo\Anaconda2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1749: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.442
Method:                 Least Squares   F-statistic:                     501.5
Date:                Wed, 07 Dec 2016   Prob (F-statistic):               0.00
Time:                        17:13:34   Log-Likelihood:            -1.4289e+05
No. Observations:               24028   AIC:                         2.859e+05
Df Residuals:                   23989   BIC:                         2.862e+05
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
host_response_rate     -10.4975      3.909     -2.685      0.007       -18.160    -2.835
accommodates            18.7163      0.656     28.520      0.000        17.430    20.003
bathrooms               70.7278      1.812     39.029      0.000        67.176    74.280
bedrooms                31.2268      1.246     25.063      0.000        28.785    33.669
beds                   -10.4369      1.026    -10.175      0.000       -12.447    -8.426
extra_people             0.2581      0.028      9.286      0.000         0.204     0.313
minimum_nights          -0.2623      0.097     -2.702      0.007        -0.453    -0.072
review_scores_rating     1.0295      0.073     14.197      0.000         0.887     1.172
f                      -13.0643     12.260     -1.066      0.287       -37.095    10.967
t                      -45.4559      9.009     -5.046      0.000       -63.114   -27.798
f                      -30.2039      7.482     -4.037      0.000       -44.868   -15.539
t                      -28.3163      7.516     -3.767      0.000       -43.048   -13.584
Apartment                4.7497     12.110      0.392      0.695       -18.987    28.487
Bed & Breakfast         -3.0598     14.333     -0.213      0.831       -31.154    25.034
Boat                   -66.2826     45.425     -1.459      0.145      -155.319    22.754
Bungalow               -42.3415     40.972     -1.033      0.301      -122.650    37.967
Cabin                 -106.0026     25.263     -4.196      0.000      -155.520   -56.485
Camper/RV              -14.2452     88.383     -0.161      0.872      -187.482   158.992
Castle                  57.8416     88.364      0.655      0.513      -115.357   231.041
Chalet                  18.7602     88.360      0.212      0.832      -154.431   191.951
Condominium             18.4188     14.663      1.256      0.209       -10.322    47.160
Dorm                    -9.0299     20.511     -0.440      0.660       -49.232    31.172
House                  -23.8991     12.258     -1.950      0.051       -47.926     0.128
Hut                     11.9183     88.521      0.135      0.893      -161.588   185.424
Igloo                  -51.5768     88.369     -0.584      0.559      -224.785   121.632
Loft                    54.8472     12.635      4.341      0.000        30.081    79.613
Other                   52.0715     17.019      3.060      0.002        18.713    85.430
Tent                    17.2791     35.341      0.489      0.625       -51.991    86.549
Townhouse                0.4810     13.359      0.036      0.971       -25.704    26.666
Villa                   21.5499     33.228      0.649      0.517       -43.580    86.680
Entire home/apt         40.4741      5.219      7.756      0.000        30.245    50.703
Private room           -35.8468      5.154     -6.955    